In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython import display

from deep_shadow import *
from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading Dataset

In [3]:
cities = ['nyc']
dates = ['summer']

zoom = 16
shadow_path = 'data/shadows/'
height_path = 'data/heights_new/'

TILES_PER_CITY = 200
BATCH_SIZE = 1

In [4]:
# keep these images off train or test dataset
path_list = ['16/19305/24641.png', 
             '16/19317/24654.png', 
             '16/19348/24629.png', 
             '16/19301/24621.png', 
             '16/19331/24653.png', 
             '16/19311/24658.png', 
             '16/19306/24625.png',
             '16/19309/24670.png',
             '16/19294/24661.png',
             '16/19329/24624.png',
             '16/19333/24635.png',
             '16/19320/24627.png',
             '16/19302/24622.png',
             '16/19342/24644.png',
             '16/19302/24621.png',
             '16/19302/24645.png']

In [5]:
train_dataset, test_dataset = get_train_test(
    height_path, shadow_path, cities, dates, zoom, TILES_PER_CITY, BATCH_SIZE, ignore_images = path_list)

### Latitude and date inclusion

In [6]:
lat, dat = False, False

### Define the generator architecture

In [7]:
def get_generator_arch():
    down_stack = [
            downsample(64, 4, apply_batchnorm=False),  # (batch_size, 256, 256, 64)
            downsample(128, 4),  # (batch_size, 128, 128, 128)
            downsample(256, 4),  # (batch_size, 64, 64, 256)
            downsample(512, 4),  # (batch_size, 32, 32, 512)
            downsample(512, 4),  # (batch_size, 16, 16, 512)
            downsample(512, 4),  # (batch_size, 8, 8, 512)
            downsample(512, 4),  # (batch_size, 4, 4, 512)
            downsample(512, 4),  # (batch_size, 2, 2, 512)
            downsample(512, 4),  # (batch_size, 1, 1, 512)
        ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
        upsample(512, 4),  # (batch_size, 16, 16, 1024)
        upsample(512, 4),  # (batch_size, 16, 16, 1024)
        upsample(256, 4),  # (batch_size, 32, 32, 512)
        upsample(128, 4),  # (batch_size, 64, 64, 256)
        upsample(64, 4),  # (batch_size, 128, 128, 128)
    ]

    return down_stack, up_stack

### Generate and Store Handpicked Test Images

In [12]:
def store_images(checkpoint_name, deep_shadow, path_list, _lat=True, _dat=True):
    city, date, zoom = 'nyc', 'summer', 16

    if not os.path.exists('data/compare_models/%s/' % checkpoint_name):
        os.makedirs('data/compare_models/%s/' % checkpoint_name)

    for path in path_list:
        i, j = int(path.split('/')[1]), int(path.split('/')[2].split('.')[0])
        img_no = '%d_%d' % (i, j)
        plot_path = 'data/compare_models/%s/%s' % (checkpoint_name, img_no)
        test_on_image(deep_shadow.generator, height_path, shadow_path, city, date, zoom, i, j, path=plot_path, save=True, lat=_lat, dat=_dat)

    return

### Train models with different loss functions and save results

In [14]:
checkpoint_names = ['_l1', '_l2', '_l1_smooth_loss', '_ssim', '_sobel', '_ssim_with_sobel', '_ssim_l1_smooth_loss', '_ssim_l1_loss', '_ssim_sobel_plus_l1', '_berhu']
loss_functions = [[l1_loss], [l2_loss], [l1_smooth_loss], [ssim_loss], [sobel_loss], [ssim_loss, sobel_loss], [ssim_loss, l1_smooth_loss], [ssim_loss, l1_loss], [ssim_loss, sobel_loss, l1_loss], [berhu_loss]]

for i in range(len(checkpoint_names)):
    tf.keras.backend.clear_session()

    down_stack, up_stack = get_generator_arch()
    
    train_dataset, test_dataset = get_train_test(
    height_path, shadow_path, cities, dates, zoom, TILES_PER_CITY, BATCH_SIZE, ignore_images = path_list)

    checkpoint_name = 'test' + checkpoint_names[i]
    checkpoint_path = 'training_checkpoints/%s' % (checkpoint_name)
    deep_shadow = DeepShadow(512, 512, down_stack, up_stack, latitude=lat, date=dat, loss_funcs=loss_functions[i], type='unet', attention=False, model_name=checkpoint_name)
    # deep_shadow.fit(checkpoint_path, train_dataset, test_dataset, 40000)

    deep_shadow.restore(checkpoint_path)
    store_images(checkpoint_name, deep_shadow, path_list, _lat = lat, _dat = dat)
    df = pd.read_csv('data/compare_models/results.csv')
    rmses, maes, mses, ssims, sobels = get_metrics(test_dataset, deep_shadow.generator, latitude=lat, date=dat)
    df.loc[len(df.index)] = [checkpoint_name, np.average(maes), np.average(mses), np.average(rmses), np.average(ssims), np.average(sobels)] 
    df.to_csv('data/compare_models/results.csv', index=False)

In [ ]:
# visual artifacts!! 19302_24621, 19302_24622, etc
